In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import requests
import time
import pandas as pd
import datetime

### Сбор датасета новостей через API Alphavantage

In [3]:

def nlp_data(date, api_key='RA2WC9ADPHNM289Z'):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=CRYPTO:APT&apikey={api_key}&time_from={date}&limit=1000&sort=EARLIEST'
    r = requests.get(url)

    if r.status_code != 200:
        print(f'Error fetching data: {r.status_code}')
        return pd.DataFrame()  

    data = r.json()

    if 'feed' not in data:
        print('No "feed" key in the response.')
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае отсутствия ключа

    if not data['feed']:
        print('Feed is empty.')
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае пустого списка

    df = pd.DataFrame(data['feed'])

     # Преобразуем формат времени в стандартный формат даты
    df['date_published'] = pd.to_datetime(df['time_published'], format='%Y%m%dT%H%M%S')
    
    # Удалим старый столбец времени
    df.drop('time_published', axis=1, inplace=True)
    
    # Сортируем значения по дате
    df_sorted = df.sort_values(by='date_published')

    # Найти последнюю дату и время в полученных данных
    last_entry = df_sorted.iloc[-1]
    last_time = last_entry['date_published']

    # Преобразуем последнюю дату и время в нужный формат
    last_time_str = last_time.strftime('%Y%m%dT%H%M')

    # Приостановка перед следующим запросом
    time.sleep(3)

    # Рекурсивный вызов для получения следующих данных
    df_2 = nlp_data(last_time_str, api_key=api_key)

    # Объединяем текущие данные с полученными рекурсивно
    result = pd.concat([df_sorted, df_2], axis=0)

    return result

# Запуск функции
# a = nlp_data('20210410T0130', api_key='RA2WC9ADPHNM289Z')
a = nlp_data('20241112T2103', api_key='RA2WC9ADPHNM289Z')

print(a)


No "feed" key in the response.
Empty DataFrame
Columns: []
Index: []


In [ ]:
# Проверяем, что DataFrame не пустой
if not a.empty:
    # Определяем начальную и конечную дату в нужном формате
    start_date = a['date_published'].min().strftime('%Y%m%dT%H%M')
    end_date = a['date_published'].max().strftime('%Y%m%dT%H%M')
    
    # Формируем имя файла с использованием начальной и конечной даты
    filename = f"news_data_{start_date}_to_{end_date}.csv"
    
    # Сохраняем DataFrame в CSV-файл
    a.to_csv(filename, index=False)
    
    # Выводим сообщение об успешном сохранении
    print(f"Данные успешно сохранены в файл: {filename}")
else:
    print("DataFrame пустой. Нет данных для сохранения.")

In [6]:
pd.read_csv(r'D:\__projects__\diploma\portfolios-optimization\notebooks\news_data\btc\combined_btc_news.csv')

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date_published
0,Geopolitical Risk Returns for Global Markets,https://www.nytimes.com/2022/03/01/business/de...,[],Investors are weighing the implications of Rus...,https://static01.nyt.com/images/2022/03/01/mul...,New York Times,Dealbook,www.nytimes.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.138797,Neutral,"[{'ticker': 'NFLX', 'relevance_score': '0.0189...",2022-03-01 12:58:36
1,"Asian shares slip, oil surges again as Russia ...",https://www.moneycontrol.com/news/business/mar...,[],"As global sanctions against Moscow tighten, th...",https://images.moneycontrol.com/static-mcnews/...,Money Control,Markets,www.moneycontrol.com,"[{'topic': 'Energy & Transportation', 'relevan...",-0.031243,Neutral,"[{'ticker': 'EQNR', 'relevance_score': '0.0639...",2022-03-02 02:32:56
2,"Business Highlights: Lobbyists leaving, rate h...",https://apnews.com/article/russia-ukraine-tech...,[],"Business Highlights: Lobbyists leaving, rate h...",https://storage.googleapis.com/afs-prod/media/...,Associated Press,Business,apnews.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.174572,Somewhat-Bearish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",2022-03-02 08:00:00
3,"Millions for Crypto Start-Ups, No Real Names N...",https://www.nytimes.com/2022/03/02/technology/...,[],Investors give money to pseudonymous developer...,https://static01.nyt.com/images/2022/02/23/bus...,New York Times,Dealbook,www.nytimes.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.036470,Neutral,"[{'ticker': 'NYT', 'relevance_score': '0.0234'...",2022-03-02 10:00:25
4,How to Make ( and Keep ) Money in the Market...,https://moneymorning.com/2022/03/02/how-to-mak...,"['chris_johnson', 'Chris Johnson']",Chris Johnson shows you what to look for in or...,https://moneymorning.com/wp-content/blogs.dir/...,Money Morning,NaN,moneymorning.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.034280,Neutral,"[{'ticker': 'X', 'relevance_score': '0.112506'...",2022-03-02 11:00:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63350,"AAPL, AMZN, RDDT, MSTR, XYZ: 5 Trending Stocks...",https://www.benzinga.com/25/05/45157181/apple-...,['Shivdeep Dhaliwal'],Wall Street experienced a positive momentum as...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.234724,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.1455...",2025-05-02 02:32:21
63351,Michael Saylor Considers Competition Copying S...,https://www.benzinga.com/markets/cryptocurrenc...,['Aniket Verma'],Strategy Inc. MSTR executive chairman Michael ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.211169,Somewhat-Bullish,"[{'ticker': 'MSTR', 'relevance_score': '0.2748...",2025-05-02 03:28:45
63352,Jack Dorsey Upbeat On Block's Bitcoin Mining C...,https://www.benzinga.com/25/05/45158116/jack-d...,['Aniket Verma'],"Jack Dorsey, CEO of Block Inc. XYZ, expressed ...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.254298,Somewhat-Bullish,"[{'ticker': 'CORZQ', 'relevance_score': '0.085...",2025-05-02 04:57:19
63353,"Riot Platforms posts Q1 loss, beats revenue es...",https://cointelegraph.com/news/riot-platforms-...,['Ciaran Lyons'],Bitcoin miner Riot Platforms has reported its ...,https://s3.cointelegraph.com/uploads/2025-05/0...,Cointelegraph,NaN,cointelegraph.com,"[{'topic': 'Life Sciences', 'relevance_score':...",-0.050416,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.1368...",2025-05-02 05:29:36


In [ ]:
api_nlp.to_csv('data_api_nlp.csv', index=False)

In [ ]:
len(data_nlp[['summary', 'title']].index[data_nlp[['summary']].duplicated()]) # подсчет дубликатов

In [183]:
data_for_nlp = data_nlp[[ 'title', 'summary', 'source' , 'date_published' ]] # отсеяли ненужные столбцы

In [186]:
data_for_nlp = data_for_nlp.drop_duplicates(keep='last') #удаление дубликатолв

In [ ]:
data_for_nlp['date_published'] = pd.to_datetime(data_for_nlp['date_published'])

# Удаление времени из столбца дат, чтобы оставить только даты
data_for_nlp['date_published'] = data_for_nlp['date_published'].dt.date

#### Колонки в данных ниже:
- title - заголовк новости,
- summary - краткое содержание новости,
- source - источник новости,
- date_published - дата публикации

In [ ]:
data_for_nlp

Объединим данные группировкой по дням, т.е. новости в течение одного дня объединятся в одно наблюдение соответствующего дня,и проделаем это только  с   колонкой с содержанием новостей, так как она самая содержательная.

In [ ]:
data_brt = data_for_nlp.groupby('date_published')['summary'].apply(' '.join).reset_index()

print(result)

In [ ]:
data_brt

In [198]:
data_brt.set_index('date_published', inplace=True)

### Применение модели  bart-large-mnli  к датасету

#### Эта модель - предварительно обученный классификатор BART-Large MNLI, использует encoder предварительно обученной модели BART-Large и дообучена на наборе данных MultiNLI, который содержит 433 тысячи пар предложений, аннотированных текстовой информацией. На вход модели подается текст и гипотеза/гипотезы и для каждой она выдает число (вероятность) от 0 до 1 насколько текст соответствует гипотезе. В нашем случае взята гипотеза : "This example is bullish for Bitcoin" (аналогичный подход с такой же гипотезой использовался в https://arxiv.org/abs/2311.14759).


In [ ]:
import torch
print(torch.cuda.is_available())

In [209]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [210]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

In [1]:
candidate_labels = ['This example is bullish for Bitcoin'] # сформировали гипотезу

In [212]:
def classify_text(text):
    result = classifier(text, candidate_labels)
    return result['scores'][0]

In [213]:
data_brt['classification'] = data_brt['summary'].apply(classify_text)# Применение модели для каждого наблюдения

In [ ]:
data_brt

В столбце classification лежат оценки модели по новостям на каждый из дней

In [221]:
data_brt.to_csv('data_from_nlp.csv', index=True)